## Creating a Linear Regression Model Using ________ Data ##

Imports & Reading in Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

Data Exploration

Data Pre-Processing

Linear Regression